# FEMA Disaster Declarations Summaries
Author: Mark Bauer

In [1]:
# import packages
import duckdb
from datetime import datetime

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0



In [3]:
# data retrieved
current_date = datetime.now()
print(f"The data was retrieved on {current_date.strftime('%Y-%m-%d')}.")

The data was retrieved on 2024-10-20.


Note: This analysis uses the Federal Emergency Management Agency’s OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA's [Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

# OpenFEMA Dataset: Disaster Declarations Summaries - v2

## Dataset
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: Disaster Declarations Summaries - v2. Retrieved from https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about [OpenFEMA Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

## Dataset Description
>Disaster Declarations Summaries is a summarized dataset describing all federally declared disasters. This dataset lists all official FEMA Disaster Declarations, beginning with the first disaster declaration in 1953 and features all three disaster declaration types: major disaster, emergency, and fire management assistance. The dataset includes declared recovery programs and geographic areas (county not available before 1964; Fire Management records are considered partial due to historical nature of the dataset).

Source: [OpenFEMA Dataset: Disaster Declarations Summaries - v2](https://www.fema.gov/openfema-data-page/disaster-declarations-summaries-v2)

## Disaster Delcarations
For more information on the disaster declaration process:
- Information about disasters: https://www.fema.gov/disasters
- How a disaster is declared: https://www.fema.gov/disasters/how-declared

# Additional Resources
To examine other cool data visualizations about this data, visit FEMA's [Disaster Declarations for States and Counties](https://www.fema.gov/data-visualization/disaster-declarations-states-and-counties) page.

# Read In Data

In [4]:
# create a duckdb database connection
con = duckdb.connect()

# create disaster_declarations table from OpenFEMA parquet file
con.execute("""
    CREATE TABLE disaster_declarations
    AS FROM read_parquet('https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.parquet')
"""
)

# sanity check of table
sql = """
    SELECT *
    FROM disaster_declarations
    LIMIT 10
"""

con.sql(sql)

┌──────────────────────┬────────────────┬─────────┬───┬──────────────────────┬──────────────────────┐
│          id          │ disasterNumber │  state  │ … │     lastRefresh      │         hash         │
│       varchar        │     int16      │ varchar │   │      timestamp       │       varchar        │
├──────────────────────┼────────────────┼─────────┼───┼──────────────────────┼──────────────────────┤
│ f15a7a79-f1c3-41bb…  │           5530 │ NV      │ … │ 2024-08-27 18:22:1…  │ 5d07e7c51bb300bfbe…  │
│ 09e3f81a-5e16-4b72…  │           5529 │ OR      │ … │ 2024-08-27 18:22:1…  │ ae87cf3c6ed795015b…  │
│ 59983f89-30bf-4888…  │           5528 │ OR      │ … │ 2024-08-27 18:22:1…  │ 432cf0995c47e3895c…  │
│ 8d13ecf0-bc2f-496b…  │           5527 │ OR      │ … │ 2024-08-27 18:22:1…  │ 2f21d90cb6bc64b0d4…  │
│ 17c24d4a-49a9-4cac…  │           5526 │ CO      │ … │ 2024-08-27 18:22:1…  │ e753ba692156f389db…  │
│ f1140a27-cb85-404c…  │           5525 │ CO      │ … │ 2024-08-27 18:22:1…  │ b1f

In [5]:
# list tables and schemas
con.sql("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,memory,main,disaster_declarations,"[id, disasterNumber, state, femaDeclarationStr...","[VARCHAR, SMALLINT, VARCHAR, VARCHAR, VARCHAR,...",False


In [6]:
# count of rows
con.sql("""
    SELECT
        COUNT(*) AS count_rows
    FROM disaster_declarations
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      67168 │
└────────────┘

In [7]:
# count of columns
con.sql("""
    SELECT
        COUNT(column_name) AS count_columns
    FROM (DESCRIBE FROM disaster_declarations)
""")

┌───────────────┐
│ count_columns │
│     int64     │
├───────────────┤
│            28 │
└───────────────┘

# Examine Column Information

In [8]:
# examine column datatypes
con.sql("""
    SELECT
        column_name,
        column_type
    FROM (DESCRIBE disaster_declarations)
""").show(max_rows=30)

┌──────────────────────────┬─────────────┐
│       column_name        │ column_type │
│         varchar          │   varchar   │
├──────────────────────────┼─────────────┤
│ id                       │ VARCHAR     │
│ disasterNumber           │ SMALLINT    │
│ state                    │ VARCHAR     │
│ femaDeclarationString    │ VARCHAR     │
│ declarationType          │ VARCHAR     │
│ declarationDate          │ DATE        │
│ fyDeclared               │ SMALLINT    │
│ incidentType             │ VARCHAR     │
│ declarationTitle         │ VARCHAR     │
│ ihProgramDeclared        │ BOOLEAN     │
│ iaProgramDeclared        │ BOOLEAN     │
│ paProgramDeclared        │ BOOLEAN     │
│ hmProgramDeclared        │ BOOLEAN     │
│ incidentBeginDate        │ DATE        │
│ incidentEndDate          │ DATE        │
│ disasterCloseoutDate     │ DATE        │
│ tribalRequest            │ BOOLEAN     │
│ fipsStateCode            │ VARCHAR     │
│ fipsCountyCode           │ VARCHAR     │
│ placeCode

In [9]:
# examine column null percentage
con.sql("""
    SELECT
        column_name,
        null_percentage
    FROM (SUMMARIZE FROM disaster_declarations)
    WHERE null_percentage > 0
    ORDER BY null_percentage DESC
""").show(max_rows=30)

┌─────────────────────────┬─────────────────┐
│       column_name       │ null_percentage │
│         varchar         │  decimal(9,2)   │
├─────────────────────────┼─────────────────┤
│ lastIAFilingDate        │           71.81 │
│ designatedIncidentTypes │           71.18 │
│ disasterCloseoutDate    │           23.50 │
│ incidentEndDate         │            1.94 │
└─────────────────────────┴─────────────────┘



# Summary Statistics

In [10]:
# examine each column in sections because of large number of columns
sql = """
    SELECT *
    FROM (SUMMARIZE disaster_declarations)
"""

# slice through columns for readability
con.sql(sql).df().iloc[:15, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,00002fed-4c4b-49c0-8031-6e9749819fde,ffffdc28-ff85-43c7-91bb-f40fc4932471,65742,None,None,None,None,None,67168,0.00
1,disasterNumber,SMALLINT,1,5541,5097,2660.472323130062,1430.2827036299207,1361,3135,3661,67168,0.00
2,state,VARCHAR,AK,WY,59,None,None,None,None,None,67168,0.00
3,femaDeclarationString,VARCHAR,DR-1-GA,FM-5541-ND,5034,None,None,None,None,None,67168,0.00
4,declarationType,VARCHAR,DR,FM,3,None,None,None,None,None,67168,0.00
5,declarationDate,DATE,1953-05-02,2024-10-19,3525,None,None,None,None,None,67168,0.00
6,fyDeclared,SMALLINT,1953,2025,74,2005.2305562172462,15.004325214213514,1998,2008,2019,67168,0.00
7,incidentType,VARCHAR,Biological,Winter Storm,26,None,None,None,None,None,67168,0.00
8,declarationTitle,VARCHAR,HURRICANE EARL,ZOGG FIRE,2355,None,None,None,None,None,67168,0.00
9,ihProgramDeclared,BOOLEAN,false,true,2,None,None,None,None,None,67168,0.00


In [11]:
# slice through columns
con.sql(sql).df().iloc[15:, :]

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
15,disasterCloseoutDate,DATE,1954-06-01,2024-10-16,1905,None,None,None,None,None,67168,23.50
16,tribalRequest,BOOLEAN,false,true,2,None,None,None,None,None,67168,0.00
17,fipsStateCode,VARCHAR,01,78,59,None,None,None,None,None,67168,0.00
18,fipsCountyCode,VARCHAR,000,840,351,None,None,None,None,None,67168,0.00
19,placeCode,VARCHAR,0,99507,1319,None,None,None,None,None,67168,0.00
20,designatedArea,VARCHAR,Abbeville (County),Zuni Pueblo (Indian Reservation),2980,None,None,None,None,None,67168,0.00
21,declarationRequestNumber,VARCHAR,10,99159,4979,None,None,None,None,None,67168,0.00
22,lastIAFilingDate,DATE,1998-10-24,2025-09-30,539,None,None,None,None,None,67168,71.81
23,incidentId,VARCHAR,1999070601,98203,4263,None,None,None,None,None,67168,0.00
24,region,SMALLINT,1,10,10,5.19698368270605,2.1282054759841644,4,5,7,67168,0.00


# Preview Values
Scan table twice to preview all columns.

In [12]:
sql = """
    SELECT *
    FROM disaster_declarations
    LIMIT 5   
"""

# examine first 15 columns
con.sql(sql).df().iloc[:, :15]

,id,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate
0,f15a7a79-f1c3-41bb-8a5c-c05fbae34423,5530,NV,FM-5530-NV,FM,2024-08-12,2024,Fire,GOLD RANCH FIRE,False,False,True,True,2024-08-11,NaT
1,09e3f81a-5e16-4b72-b317-1c64e0cfa59c,5529,OR,FM-5529-OR,FM,2024-08-09,2024,Fire,LEE FALLS FIRE,False,False,True,True,2024-08-08,NaT
2,59983f89-30bf-4888-b21b-62e8d57d9aac,5528,OR,FM-5528-OR,FM,2024-08-06,2024,Fire,ELK LANE FIRE,False,False,True,True,2024-08-04,NaT
3,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0,5527,OR,FM-5527-OR,FM,2024-08-02,2024,Fire,MILE MARKER 132 FIRE,False,False,True,True,2024-08-02,NaT
4,17c24d4a-49a9-4cac-9322-e5427c4cdfeb,5526,CO,FM-5526-CO,FM,2024-08-01,2024,Fire,QUARRY FIRE,False,False,True,True,2024-07-30,NaT


In [13]:
# examine last 15 columns
con.sql(sql).df().iloc[:, 15:]

,disasterCloseoutDate,tribalRequest,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash
0,NaT,False,32,031,99031,Washoe (County),24123,NaT,2024081201,9,R,2024-08-27 18:22:14.800,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd
1,NaT,False,41,067,99067,Washington (County),24122,NaT,2024081001,10,R,2024-08-27 18:22:14.800,ae87cf3c6ed795015b714af7166c7c295b2b67c7
2,NaT,False,41,031,99031,Jefferson (County),24116,NaT,2024080701,10,R,2024-08-27 18:22:14.800,432cf0995c47e3895cea696ede5621b810460501
3,NaT,False,41,017,99017,Deschutes (County),24111,NaT,2024080301,10,R,2024-08-27 18:22:14.800,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa
4,NaT,False,08,059,99059,Jefferson (County),24106,NaT,2024080102,8,R,2024-08-27 18:22:14.800,e753ba692156f389dbe19f7a1c332d04ae145f74


In [14]:
# count duplicate IDs
con.sql("""
    SELECT
        id,
        COUNT(id) AS count
    FROM disaster_declarations
    GROUP BY id
    HAVING count > 1
""")

┌─────────┬───────┐
│   id    │ count │
│ varchar │ int64 │
├─────────┴───────┤
│     0 rows      │
└─────────────────┘

In [15]:
# latest record refreshed date
sql = """
    SELECT
        lastRefresh
    FROM disaster_declarations
    ORDER BY lastRefresh DESC
    LIMIT 1
"""

record_last_updated = (
    con
    .sql(sql)
    .df()
    .loc[:, ['lastRefresh']]
    .values[0][0]
)

print(f"Record last updated at: {record_last_updated}")

Record last updated at: 2024-10-19T20:05:02.440000000


In [16]:
# top 5 earliest declared disaster event
sql = """
    SELECT
        declarationDate,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        incidentBeginDate,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY declarationDate ASC
    LIMIT 5
"""

con.sql(sql).df()

,declarationDate,disasterNumber,state,femaDeclarationString,declarationType,incidentBeginDate,incidentType,declarationTitle
0,1953-05-02,1,GA,DR-1-GA,DR,1953-05-02,Tornado,TORNADO
1,1953-05-15,2,TX,DR-2-TX,DR,1953-05-15,Tornado,TORNADO & HEAVY RAINFALL
2,1953-05-29,3,LA,DR-3-LA,DR,1953-05-29,Flood,FLOOD
3,1953-06-02,4,MI,DR-4-MI,DR,1953-06-02,Tornado,TORNADO
4,1953-06-06,5,MT,DR-5-MT,DR,1953-06-06,Flood,FLOODS


In [17]:
# top 5 latest disaster declared event
sql = """
    SELECT
        declarationDate,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        incidentBeginDate,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY declarationDate DESC
    LIMIT 5
"""

con.sql(sql).df()

,declarationDate,disasterNumber,state,femaDeclarationString,declarationType,incidentBeginDate,incidentType,declarationTitle
0,2024-10-19,4837,NC,DR-4837-NC,DR,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
1,2024-10-19,4837,NC,DR-4837-NC,DR,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
2,2024-10-19,4837,NC,DR-4837-NC,DR,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
3,2024-10-19,4837,NC,DR-4837-NC,DR,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
4,2024-10-16,4836,AK,DR-4836-AK,DR,2024-08-05,Flood,FLOODING


In [18]:
# top 5 latest incident begin date
sql = """
    SELECT
        incidentBeginDate,
        lastRefresh,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY incidentBeginDate DESC
    LIMIT 5
"""

con.sql(sql).df()

,incidentBeginDate,lastRefresh,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,incidentType,declarationTitle
0,2024-10-05,2024-10-07 14:01:33.043,3622,FL,EM-3622-FL,EM,2024-10-07,Hurricane,HURRICANE MILTON
1,2024-10-05,2024-10-07 14:01:33.043,3622,FL,EM-3622-FL,EM,2024-10-07,Hurricane,HURRICANE MILTON
2,2024-10-05,2024-10-07 14:01:33.043,3622,FL,EM-3622-FL,EM,2024-10-07,Hurricane,HURRICANE MILTON
3,2024-10-05,2024-10-07 14:01:33.043,3622,FL,EM-3622-FL,EM,2024-10-07,Hurricane,HURRICANE MILTON
4,2024-10-05,2024-10-07 14:01:33.043,3622,FL,EM-3622-FL,EM,2024-10-07,Hurricane,HURRICANE MILTON


In [19]:
# top 5 event last refreshed
sql = """
    SELECT
        lastRefresh,
        disasterNumber,
        state,
        femaDeclarationString,
        declarationType,
        declarationDate,
        incidentBeginDate,
        incidentType,
        declarationTitle
    FROM disaster_declarations
    ORDER BY lastRefresh DESC
    LIMIT 5
"""

con.sql(sql).df()

,lastRefresh,disasterNumber,state,femaDeclarationString,declarationType,declarationDate,incidentBeginDate,incidentType,declarationTitle
0,2024-10-19 20:05:02.440,4837,NC,DR-4837-NC,DR,2024-10-19,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
1,2024-10-19 20:05:02.440,4837,NC,DR-4837-NC,DR,2024-10-19,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
2,2024-10-19 20:05:02.440,4837,NC,DR-4837-NC,DR,2024-10-19,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
3,2024-10-19 20:05:02.440,4837,NC,DR-4837-NC,DR,2024-10-19,2024-09-16,Tropical Storm,POTENTIAL TROPICAL CYCLONE EIGHT
4,2024-10-16 22:22:43.754,4836,AK,DR-4836-AK,DR,2024-10-16,2024-08-05,Flood,FLOODING


In [20]:
# count per declaration
sql = """
    SELECT
       femaDeclarationString,
       count(id) AS count
    FROM disaster_declarations
    GROUP BY femaDeclarationString
    ORDER BY count DESC   
    LIMIT 20
"""

con.sql(sql).df()

,femaDeclarationString,count
0,DR-4522-ME,443
1,EM-3458-TX,257
2,DR-1239-TX,257
3,DR-4485-TX,257
4,EM-3261-TX,255
5,DR-1606-TX,254
6,EM-3216-TX,254
7,DR-4586-TX,254
8,DR-1624-TX,254
9,EM-3554-TX,254


One declaration has many records. Examine further.

In [21]:
# examine count unique for femaDeclarationString = 'DR-4522-ME'
(con
 .sql("""
    SELECT *
    FROM disaster_declarations
    WHERE femaDeclarationString = 'DR-4522-ME'
""")
 .df()
 .describe(include='object')
 .loc['unique']
)

id                          443
state                         1
femaDeclarationString         1
declarationType               1
incidentType                  1
declarationTitle              1
fipsStateCode                 1
fipsCountyCode               17
placeCode                   443
designatedArea              438
declarationRequestNumber      1
incidentId                    1
designatedIncidentTypes       0
hash                        443
Name: unique, dtype: object

The duplicate records for the same declaration suggest this dataset is grouped by **declaration** and **places** (i.e. placeCode) that received the declaration. Let's examine only unique declarations.

**Table xx.** Count of Disasters Declared

In [22]:
sql = """
    SELECT
        COUNT(DISTINCT femaDeclarationString) AS disasters_declared,
        strftime(MIN(declarationDate), '%Y-%m-%d') AS date_min,
        strftime(MAX(lastRefresh), '%Y-%m-%d') AS date_max
    FROM disaster_declarations  
"""

# push to pandas df
df = con.sql(sql).df()

# capture values
count = df['disasters_declared'].values[0]
date_min = df['date_min'].values[0]
date_max = df['date_max'].values[0]

print(f"{count:,} disasters declared between {date_min} and {date_max}.")

4,993 disasters declared between 1953-05-02 and 2024-10-19.


**Table xx.** Number of Disasters Declared by Declaration Type

In [23]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            declarationType,
            id           
        FROM disaster_declarations
    )
    
    SELECT
       declarationType,
       count(id) AS count
    FROM declarations_unique
    GROUP BY declarationType
    ORDER BY count DESC   
"""

con.sql(sql)

┌─────────────────┬───────┐
│ declarationType │ count │
│     varchar     │ int64 │
├─────────────────┼───────┤
│ DR              │  2830 │
│ FM              │  1540 │
│ EM              │   623 │
└─────────────────┴───────┘

Declaraton Type: Two character code that defines if the disaster is a Major Disaster Declaration (DR), Emergency Declaration (EM), or Fire Management Assistance Declaration (FM).

# Analysis

**Table xx.** Number of Disaster Declarations by Incident Type

In [24]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            incidentType,
            id           
        FROM disaster_declarations
    )
    
    SELECT
       incidentType,
       count(id) AS count
    FROM declarations_unique
    GROUP BY incidentType
    ORDER BY count DESC  
"""

con.sql(sql).df()

,incidentType,count
0,Fire,1639
1,Severe Storm,1093
2,Flood,899
3,Hurricane,452
4,Tornado,182
5,Snowstorm,171
6,Biological,167
7,Severe Ice Storm,74
8,Typhoon,58
9,Drought,46


**Table xx.** Top 20 Years with the Highest Number of Disaster Declarations

In [25]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            declarationDate,
            id           
        FROM disaster_declarations
    )
    
    SELECT
        strftime(declarationDate, '%Y') AS year,
        count(id) AS count
    FROM declarations_unique
    GROUP BY year
    ORDER BY count DESC
    LIMIT 20
"""

con.sql(sql)

┌─────────┬───────┐
│  year   │ count │
│ varchar │ int64 │
├─────────┼───────┤
│ 2020    │   315 │
│ 2011    │   242 │
│ 2024    │   160 │
│ 1996    │   158 │
│ 2005    │   155 │
│ 2008    │   143 │
│ 2006    │   143 │
│ 2017    │   137 │
│ 2007    │   136 │
│ 1998    │   128 │
│ 2018    │   124 │
│ 2003    │   123 │
│ 2021    │   120 │
│ 2004    │   118 │
│ 2002    │   118 │
│ 2009    │   115 │
│ 2023    │   114 │
│ 2000    │   114 │
│ 2012    │   112 │
│ 1999    │   110 │
├─────────┴───────┤
│     20 rows     │
└─────────────────┘

**Table xx.** Top 20 States with the Highest Number of Disaster Declarations

In [26]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            state,
            id           
        FROM disaster_declarations
    )
    
    SELECT
       state,
       count(id) AS count
    FROM declarations_unique
    GROUP BY state
    ORDER BY count DESC   
    LIMIT 20
"""

con.sql(sql)

┌─────────┬───────┐
│  state  │ count │
│ varchar │ int64 │
├─────────┼───────┤
│ CA      │   384 │
│ TX      │   376 │
│ OK      │   230 │
│ WA      │   208 │
│ FL      │   185 │
│ OR      │   158 │
│ NM      │   118 │
│ AZ      │   116 │
│ NY      │   116 │
│ LA      │   108 │
│ NV      │   106 │
│ CO      │   105 │
│ MT      │   104 │
│ AL      │   102 │
│ MS      │    94 │
│ TN      │    93 │
│ SD      │    92 │
│ KY      │    90 │
│ KS      │    88 │
│ AK      │    84 │
├─────────┴───────┤
│     20 rows     │
└─────────────────┘

**Table xx.** Top 20 States, Years, and Incident Types with the Highest Number of Disaster Declarations

In [27]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            state,
            declarationDate,
            incidentType,
            id           
        FROM disaster_declarations
    )
    
    SELECT
       state,
       strftime(declarationDate, '%Y') AS year,
       incidentType,
       count(id) AS count
    FROM declarations_unique
    GROUP BY state, year, incidentType
    ORDER BY count DESC   
    LIMIT 20
"""

con.sql(sql)

┌─────────┬─────────┬──────────────┬───────┐
│  state  │  year   │ incidentType │ count │
│ varchar │ varchar │   varchar    │ int64 │
├─────────┼─────────┼──────────────┼───────┤
│ TX      │ 2011    │ Fire         │    57 │
│ TX      │ 1996    │ Fire         │    55 │
│ TX      │ 1998    │ Fire         │    31 │
│ CA      │ 2017    │ Fire         │    25 │
│ OK      │ 2011    │ Fire         │    25 │
│ OK      │ 2006    │ Fire         │    23 │
│ TX      │ 2008    │ Fire         │    23 │
│ CA      │ 2020    │ Fire         │    21 │
│ CA      │ 2004    │ Fire         │    21 │
│ TX      │ 2006    │ Fire         │    20 │
│ CO      │ 2002    │ Fire         │    19 │
│ CA      │ 2007    │ Fire         │    19 │
│ CA      │ 2008    │ Fire         │    18 │
│ OR      │ 2020    │ Fire         │    18 │
│ CA      │ 2018    │ Fire         │    17 │
│ OK      │ 2020    │ Biological   │    17 │
│ TX      │ 2000    │ Fire         │    17 │
│ CA      │ 2003    │ Fire         │    17 │
│ CA      

**Table xx.** Top 20 States, Places, Designated Areas, Years, and Incident Types with the Highest Number of Disaster Declarations

In [28]:
sql = """
    SELECT
        state,
        placeCode,
        designatedArea,
        strftime(declarationDate, '%Y') AS year,
        incidentType,
        count(id) AS count
    FROM disaster_declarations
    WHERE placeCode != 0
    GROUP BY ALL
    ORDER BY count DESC  
    LIMIT 20
"""

con.sql(sql)

┌─────────┬───────────┬───────────────────────────────┬─────────┬──────────────┬───────┐
│  state  │ placeCode │        designatedArea         │  year   │ incidentType │ count │
│ varchar │  varchar  │            varchar            │ varchar │   varchar    │ int64 │
├─────────┼───────────┼───────────────────────────────┼─────────┼──────────────┼───────┤
│ CA      │ 99037     │ Los Angeles (County)          │ 2007    │ Fire         │     8 │
│ OK      │ 99109     │ Oklahoma (County)             │ 2011    │ Fire         │     7 │
│ LA      │ 99015     │ Bossier (Parish)              │ 2020    │ Hurricane    │     6 │
│ LA      │ 99069     │ Natchitoches (Parish)         │ 2020    │ Hurricane    │     6 │
│ LA      │ 99075     │ Plaquemines (Parish)          │ 2020    │ Hurricane    │     6 │
│ LA      │ 99081     │ Red River (Parish)            │ 2020    │ Hurricane    │     6 │
│ LA      │ 99087     │ St. Bernard (Parish)          │ 2020    │ Hurricane    │     6 │
│ LA      │ 99095    

**Table xx.** Number of Disaster Declarations requested by a Tribal Nation

In [29]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            tribalRequest,
            id           
        FROM disaster_declarations
    )
    
    SELECT
       tribalRequest,
       count(id) AS count
    FROM declarations_unique
    GROUP BY tribalRequest
    ORDER BY count DESC  
"""

con.sql(sql)

┌───────────────┬───────┐
│ tribalRequest │ count │
│    boolean    │ int64 │
├───────────────┼───────┤
│ false         │  4885 │
│ true          │   108 │
└───────────────┴───────┘

**Table xx.** Top 20 States and Incident Types with the Highest Number of Disaster Declarations requested by a Tribal Nation

In [30]:
sql = """
    WITH declarations_unique AS (
    
        SELECT
            DISTINCT ON (femaDeclarationString) femaDeclarationString,
            state,
            incidentType,
            id           
        FROM disaster_declarations
        WHERE tribalRequest = true
    )
    
    SELECT
       state,
       incidentType,
       count(id) AS count
    FROM declarations_unique
    GROUP BY state, incidentType
    ORDER BY count DESC  
    LIMIT 20
"""

con.sql(sql).df()

,state,incidentType,count
0,OK,Biological,15
1,NM,Biological,11
2,CA,Severe Storm,9
3,FL,Hurricane,5
4,AZ,Severe Storm,4
5,NE,Biological,4
6,OK,Severe Storm,4
7,SD,Biological,3
8,KS,Biological,3
9,FL,Biological,2


# Export Data
Export data to reproduce.

In [31]:
# export to parquet
con.sql("COPY disaster_declarations TO 'data/disaster-declarations.parquet' (FORMAT PARQUET);")

In [32]:
# sanity check
%ls data/

disaster-declarations.parquet  public-assistance.parquet


In [33]:
# sanity check on exported parquet file
con.sql("SELECT * FROM read_parquet('data/disaster-declarations.parquet') LIMIT 10;")

┌──────────────────────┬────────────────┬─────────┬───┬──────────────────────┬──────────────────────┐
│          id          │ disasterNumber │  state  │ … │     lastRefresh      │         hash         │
│       varchar        │     int16      │ varchar │   │      timestamp       │       varchar        │
├──────────────────────┼────────────────┼─────────┼───┼──────────────────────┼──────────────────────┤
│ f15a7a79-f1c3-41bb…  │           5530 │ NV      │ … │ 2024-08-27 18:22:1…  │ 5d07e7c51bb300bfbe…  │
│ 09e3f81a-5e16-4b72…  │           5529 │ OR      │ … │ 2024-08-27 18:22:1…  │ ae87cf3c6ed795015b…  │
│ 59983f89-30bf-4888…  │           5528 │ OR      │ … │ 2024-08-27 18:22:1…  │ 432cf0995c47e3895c…  │
│ 8d13ecf0-bc2f-496b…  │           5527 │ OR      │ … │ 2024-08-27 18:22:1…  │ 2f21d90cb6bc64b0d4…  │
│ 17c24d4a-49a9-4cac…  │           5526 │ CO      │ … │ 2024-08-27 18:22:1…  │ e753ba692156f389db…  │
│ f1140a27-cb85-404c…  │           5525 │ CO      │ … │ 2024-08-27 18:22:1…  │ b1f